# Exploring conformational space of selected macrocycles - notes

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [19]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Fri Mar 24 11:54:02 2017


## "M1" macrocycle

The crystallographic structure of "M1" (shown below) was used as a starting geometry for generating various conformers.

In [20]:
cm1 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.xyz','r').read()
vcm1 = py3Dmol.view(width=400,height=400)
vcm1.addModel(cm1,'xyz')
vcm1.setStyle({'stick':{}})
vcm1.setBackgroundColor('0xeeeeee')
vcm1.zoomTo()
vcm1.show()

### Conformers generated with the Balloon software:

In [21]:
inps_m1_balloon = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_crystal/*.sdf')
n = len(inps_m1_balloon)
n

6

In [22]:
# grep energies
e_m1 = []
for inp in inps_m1_balloon:
    with open(inp,'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if "M  END" in line:
                e_m1.append(float(lines[i+1]))
e_m1

[53.24562081155721,
 53.58376642026054,
 57.7600582967278,
 62.42446771209878,
 55.44127295167746,
 53.65138540775401]

Conformers (from the lowest energy):

In [23]:
%%html
<table>
  <tr>
    <td id="m1_b1" ></td>
    <td id="m1_b2" ></td>
    <td id="m1_b3" ></td>   
  </tr>
  <tr>
    <td> m1_b1, E = </td>
    <td> m1_b2, E = </td>
    <td> m1_b3, E = </td>   
  </tr>
  <tr>
    <td id="m1_b4" ></td>    
    <td id="m1_b5" ></td>
    <td id="m1_b6" ></td>   
  </tr>
  <tr>
    <td> m1_b4, E = </td>
    <td> m1_b5, E = </td>
    <td> m1_b6, E = </td>   
  </tr>
</table>

,,
"m1_b1, E =","m1_b2, E =","m1_b3, E ="
,,
"m1_b4, E =","m1_b5, E =","m1_b6, E ="


In [57]:
p1_b_handles=[]
for inp in inps_m1_balloon:
    f1_b = open(inp, 'r').read()
    p1_b = py3Dmol.view(width=200,height=200)
    p1_b.addModel(f1_b,'sdf')
    p1_b.setStyle({'stick':{}})
    p1_b.setBackgroundColor('0xeeeeee')
    p1_b.zoomTo()
    p1_b_handles.append(p1_b)

In [25]:
p1_b_handles[0].insert('m1_b1')

In [26]:
p1_b_handles[1].insert('m1_b2')

In [27]:
p1_b_handles[2].insert('m1_b3')

In [28]:
p1_b_handles[3].insert('m1_b4')

In [29]:
p1_b_handles[4].insert('m1_b5')

In [30]:
p1_b_handles[5].insert('m1_b6')

We can now align all these structures against the crystal structure:

In [85]:
templ = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.sdf')
mol1  = [x for x in templ if x is not None]
suppl = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_out.sdf')
mol2  = [x for x in suppl if x is not None]
print(mol1)
print(mol2)

AllChem.AlignMolConformers(mol2)

[<rdkit.Chem.rdchem.Mol object at 0x7f650cf8e710>, <rdkit.Chem.rdchem.Mol object at 0x7f650cf8e4e0>, <rdkit.Chem.rdchem.Mol object at 0x7f650cf8e8a0>, <rdkit.Chem.rdchem.Mol object at 0x7f650cf8e8f0>, <rdkit.Chem.rdchem.Mol object at 0x7f650cf8e940>, <rdkit.Chem.rdchem.Mol object at 0x7f650cf8e990>]


ArgumentError: Python argument types in
    rdkit.Chem.rdMolAlign.AlignMolConformers(list)
did not match C++ signature:
    AlignMolConformers(RDKit::ROMol {lvalue} mol, boost::python::api::object atomIds=[], boost::python::api::object confIds=[], boost::python::api::object weights=[], bool reflect=False, unsigned int maxIters=50, boost::python::api::object RMSlist=None)

In [15]:
mb = Chem.MolToMolBlock(list_all)


AllChem.AlignMolConformers(m1_all)

p = py3Dmol.view(width=400,height=400)
for conf in m1_all:
    mb = Chem.MolToMolBlock(conf,confId=conf.GetId())
    p.addModel(mb,'sdf')
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

ArgumentError: Python argument types in
    rdkit.Chem.rdMolAlign.AlignMolConformers(NoneType)
did not match C++ signature:
    AlignMolConformers(RDKit::ROMol {lvalue} mol, boost::python::api::object atomIds=[], boost::python::api::object confIds=[], boost::python::api::object weights=[], bool reflect=False, unsigned int maxIters=50, boost::python::api::object RMSlist=None)

In [58]:
m = Chem.AddHs(Chem.MolFromSmiles('COc1ccc2[C@H](O)[C@@H](COc2c1)N3CCC(O)(CC3)c4ccc(F)cc4'))
print(m)
AllChem.EmbedMultipleConfs(m,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)
mb = Chem.MolToMolBlock(m)


In [32]:
core = m.GetSubstructMatch(Chem.MolFromSmiles('C1C(O)c2ccccc2OC1'))
print(core)
m

(8, 6, 7, 5, 4, 3, 2, 12, 11, 10, 9)


In [33]:

AllChem.AlignMolConformers(m,atomIds=core)

p = py3Dmol.view(width=400,height=400)
for conf in m.GetConformers():
    mb = Chem.MolToMolBlock(m,confId=conf.GetId())
    p.addModel(mb,'sdf')
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()



### Conformers generated with the RDKit software:

In [ ]:
inps_m1_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_sdf/*.sdf')
n = len(inps_m1_rdkit)
print(n)

In [ ]:
%%html
<table>
  <tr>
    <td id="a1" ></td>
    <td id="a2" ></td>
  </tr>
  <tr>
    <td> p1 </td>
    <td> p2 </td>
  </tr>
</table>

## "M7" macrocycle

start: crystal geometry

In [ ]:
cm7 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m7/m7_crystal.sdf','r').read()
vcm7 = py3Dmol.view(width=400,height=400)
vcm7.addModel(cm7,'xyz')
vcm7.setStyle({'stick':{}})
vcm7.setBackgroundColor('0xeeeeee')
vcm7.zoomTo()
vcm7.show()

### Conformers generated with the Balloon software:

In [ ]:
inps_m7_balloon = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/results_crystal/*.sdf')
n = len(inps_m7_balloon)
print(n)

In [ ]:
# grep energies
e_m7 = []
for inp in inps_m7_balloon:
    with open(inp,'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if "M  END" in line:
                e_m7.append(float(lines[i+1]))
e_m7

In [ ]:
%%html
<table>
  <tr>
    <td id="m7_b1" ></td>
    <td id="m7_b2" ></td>
    <td id="m7_b3" ></td> 
    <td id="m7_b4" ></td>    
  </tr>
  <tr>
    <td> m7_b1, E = </td>
    <td> m7_b2, E = </td>
    <td> m7_b3, E = </td>   
    <td> m7_b4, E = </td>       
  </tr>
  <tr>
    <td id="m7_b5" ></td>    
    <td id="m7_b6" ></td>
    <td id="m7_b7" ></td>   
  </tr>
  <tr>
    <td> m7_b5, E = </td>
    <td> m7_b6, E = </td>
    <td> m7_b7, E = </td>   
  </tr>
  <tr>
    <td id="m7_b8" ></td>    
    <td id="m7_b9" ></td>
    <td id="m7_b10" ></td>   
  </tr>
  <tr>
    <td> m7_b8, E = </td>
    <td> m7_b9, E = </td>
    <td> m7_b10, E = </td>   
  </tr>
</table>

In [ ]:
p7_b_handles=[]
for inp in inps_m7_balloon:
    f7_b = open(inp, 'r').read()
    p7_b = py3Dmol.view(width=200,height=200)
    p7_b.addModel(f7_b,'sdf')
    p7_b.setStyle({'stick':{}})
    p7_b.setBackgroundColor('0xeeeeee')
    p7_b.zoomTo()
    p7_b_handles.append(p7_b)

In [ ]:
p7_b_handles[0].insert('m7_b1')

In [ ]:
p7_b_handles[1].insert('m7_b2')

In [ ]:
p7_b_handles[2].insert('m7_b3')

In [ ]:
p7_b_handles[3].insert('m7_b4')

In [ ]:
p7_b_handles[4].insert('m7_b5')

In [ ]:
p7_b_handles[5].insert('m7_b6')

In [ ]:
p7_b_handles[6].insert('m7_b7')

In [ ]:
p7_b_handles[7].insert('m7_b8')

In [ ]:
p7_b_handles[8].insert('m7_b9')

In [ ]:
p7_b_handles[9].insert('m7_b10')

We can now align all these structures against the crystal structure: